# DSCI 525 - Web and Cloud Computing

## Group 14
- 	Nishar	Dhruvi	
- 	Ouedraogo Flora
- 	Hong	Eyre
- 	Kohli	Manvir

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [2]:


# change to the directory that you want to download
%cd /Users/floraouedraogo/Documents/DSCI/DSCI 525/figshareexp

/Users/floraouedraogo/Documents/DSCI/DSCI 525/figshareexp


In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files


[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:

%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.68 s, sys: 7.36 s, total: 11 s
Wall time: 7min 50s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.81 s, sys: 928 ms, total: 8.73 s
Wall time: 9.7 s


In [7]:

%ls -ltr figsharerainfall/

total 12428672
drwxr-xr-x@ 30 floraouedraogo  staff        960 Mar 30 00:18 __MACOSX/
-rw-r--r--   1 floraouedraogo  staff  814041183 Mar 30 20:17 data.zip
-rw-r--r--   1 floraouedraogo  staff   95376895 Mar 30 20:17 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff   94960113 Mar 30 20:17 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff   82474546 Mar 30 20:17 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  127613760 Mar 30 20:17 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  232118894 Mar 30 20:17 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  330360682 Mar 30 20:17 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  254009247 Mar 30 20:17 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  235661418 Mar 30 20:17 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  294260911 Mar 30 20:17 BCC-CSM2-MR_d

Combining data CSVs

In [8]:
%%time
import pandas as pd
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("combined_data.csv")

CPU times: user 3min 21s, sys: 9.75 s, total: 3min 31s
Wall time: 3min 35s


In [11]:
%%time
# Reading all at once
df = pd.read_csv("combined_data.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
CMCC-CM2-SR5        3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
observed              46020
Name: model, dtype: int64
CPU times: user 30.9 s, sys: 7.5 s, total: 38.4 s
Wall time: 42.9 s


In [12]:
%%time
# Reading specific column only
use_cols = ['model']
df = pd.read_csv("combined_data.csv",usecols=use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
CMCC-CM2-SR5        3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
observed              46020
Name: model, dtype: int64
CPU times: user 17.4 s, sys: 1.68 s, total: 19 s
Wall time: 19.6 s


In [14]:
%%time
df = pd.read_csv("combined_data.csv")
print(f"Memory usage with rain as float64 and time as object: {df.memory_usage().sum() / 1e6:.2f} MB")
df['rain (mm/day)'] = df['rain (mm/day)'].astype('float32')
df['time'] = pd.to_datetime(df['time']).dt.date
print(f"Memory usage with rain as float32 and time as datetime object: {df.memory_usage().sum() / 1e6:.2f} MB")
print(df["model"].value_counts())

Memory usage with rain as float64 and time as object: 3500.78 MB
Memory usage with rain as float32 and time as datetime object: 3250.72 MB
MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
CMCC-CM2-SR5        3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
observed              46020
Name: model, dtype: int64
CPU times: user 43.5 s, sys: 10.7 s, total: 54.2 s


In [15]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
observed              46020
dtype: int64
CPU times: user 30.9 s, sys: 4.8 s, total: 35.7 s
Wall time: 38 s


In [16]:
%%time

use_cols = ['time','rain (mm/day)','model']

chunks = []

for chunk in pd.read_csv("combined_data.csv", chunksize=10_000_000,usecols = use_cols):
    chunks.append(chunk)

df = pd.concat(chunks)

print(f"Memory usage with rain as float64 and time as object: {df.memory_usage().sum() / 1e6:.2f} MB")
df['rain (mm/day)'] = df['rain (mm/day)'].astype('float32')
df['time'] = pd.to_datetime(df['time']).dt.date
print(f"Memory usage with rain as float32 and time as datetime object: {df.memory_usage().sum() / 1e6:.2f} MB")
print(df['model'].value_counts())

Memory usage with rain as float64 and time as object: 1500.33 MB
Memory usage with rain as float32 and time as datetime object: 1250.28 MB
MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
CMCC-CM2-SR5        3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
observed              46020
Name: model, dtype: int64
CPU times: user 40.5 s, sys: 9.19 s, total: 49.6 s
